In [141]:
import pandas as pd

C = pd.read_stata(
    "/Users/clarissaache/Downloads/microdata_2005_06_social_stata/sec 2a.dta", convert_categoricals=False
)
print(C.shape)
C.sample(5)


(99315, 29)


,hhcode,province,region,psu,idc,s2aq01,s2aq02,s2aq03,s2bq01,s2bq02,...,s2bq12,s2bq13,s2bq14,s2bq15,s2bq16,s2bq17,s2bq18,s2bq19a,s2bq19b,s2bq19c
74541,3.082010e+09,3,2,30820101,55,NaN,NaN,NaN,1,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
7280,1.052030e+09,1,2,10520301,2,1.0,1.0,1.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
8871,1.062030e+09,1,2,10620301,2,1.0,1.0,1.0,3,NaN,...,NaN,5.0,13.0,2.0,NaN,5.0,1.0,1300.0,100.0,1400.0
59477,2.112040e+09,2,2,21120402,51,1.0,1.0,1.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
16678,1.111240e+09,1,1,11112402,52,1.0,1.0,1.0,3,NaN,...,1.0,4.0,6.0,1.0,5.0,5.0,1.0,8400.0,1000.0,9400.0


In [142]:
# example of ONE HOUSEHOLD (idc separates individuals within households)
C.loc[C['hhcode']==1011010105]

,hhcode,province,region,psu,idc,s2aq01,s2aq02,s2aq03,s2bq01,s2bq02,...,s2bq12,s2bq13,s2bq14,s2bq15,s2bq16,s2bq17,s2bq18,s2bq19a,s2bq19b,s2bq19c
16,1.011010e+09,1,1,10110101,1,1.0,1.0,1.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
17,1.011010e+09,1,1,10110101,2,1.0,1.0,1.0,3,NaN,...,1.0,5.0,5.0,1.0,4.0,5.0,1.0,1800.0,2000.0,3800.0
18,1.011010e+09,1,1,10110101,3,1.0,1.0,1.0,3,NaN,...,1.0,5.0,4.0,1.0,3.0,5.0,1.0,1800.0,2000.0,3800.0
19,1.011010e+09,1,1,10110101,51,1.0,1.0,1.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
20,1.011010e+09,1,1,10110101,52,1.0,1.0,1.0,3,NaN,...,1.0,5.0,8.0,1.0,7.0,5.0,1.0,0.0,0.0,1250.0
21,1.011010e+09,1,1,10110101,53,1.0,1.0,1.0,3,NaN,...,1.0,5.0,6.0,1.0,5.0,5.0,1.0,0.0,0.0,800.0


In [143]:
# Create a yes/no column for "ever admitted"
C['ever_admitted']= 'yes'
C.loc[(C["s2bq01"] == 1), 'ever_admitted'] = 'no'
C['currently_enrolled']= 'no'
C.loc[(C["s2bq01"] == 3), 'currently_enrolled'] = 'yes'

In [144]:
# Reduce to the questions of interest (4 questions):

C["max_level_acheived"] = C["s2bq05"]
C["why_not"] = C["s2bq02"]
#C['enrolled_last_year']=C["s2bq05"]

C_relevant = C[
    [
        "hhcode",
        "idc",
        "ever_admitted",
        "max_level_acheived",
        "currently_enrolled",
        "why_not",
        "region",
        "province",
    ]
].copy()


In [145]:
C_relevant

,hhcode,idc,ever_admitted,max_level_acheived,currently_enrolled,why_not,region,province
0,1.011010e+09,1,no,NaN,no,NaN,1,1
1,1.011010e+09,51,yes,17.0,no,NaN,1,1
2,1.011010e+09,1,yes,12.0,no,NaN,1,1
3,1.011010e+09,51,yes,16.0,no,NaN,1,1
4,1.011010e+09,52,yes,NaN,yes,NaN,1,1
...,...,...,...,...,...,...,...,...
99310,4.071340e+09,52,yes,NaN,yes,NaN,1,4
99311,4.071340e+09,1,yes,8.0,no,NaN,1,4
99312,4.071340e+09,51,no,NaN,no,NaN,1,4
99313,4.071340e+09,52,yes,NaN,yes,NaN,1,4


In [146]:
B = pd.read_stata(
    "/Users/clarissaache/Downloads/microdata_2005_06_social_stata//1a_roster.dta"
)
B.sample(5)

,hhcode,idc,s1aq02,s1aq03,s1aq04,age,s1aq05a,s1aq05b,s1aq05c,s1aq06,s1aq07,s1aq08,s1aq09,s1aq10,region,province,psu
92940,3.232020e+09,56,son/daughter,male,1,2,1.0,1.0,2003.0,unmarried/never married,NaN,1,51,yes,rural,nwfp,32320201
52031,2.051130e+09,4,grand child,male,1,14,0.0,0.0,1991.0,unmarried/never married,NaN,2,52,yes,urban,sindh,20511302
37376,1.282020e+09,53,father/mother,female,1,72,0.0,0.0,1932.0,widow/widower,NaN,not alive,not alive,yes,rural,punjab,12820201
78323,3.052040e+09,58,son/daughter in law,female,1,19,0.0,9.0,1987.0,currently married,2.0,not in roster,not in roster,yes,rural,nwfp,30520401
56631,2.052040e+09,1,head,male,1,35,0.0,0.0,1970.0,currently married,51.0,not alive,not alive,yes,rural,sindh,20520401


In [147]:
# Change the names of the questions for readability
B['sex'] = B['s1aq03']
B['marital_status'] = B['s1aq06']

B_relevant = B[['hhcode','idc', 'age', 'sex', 'marital_status']].copy()

In [148]:
# Made the Household Code column a str type
B["hhcode"] = B["hhcode"].astype("str")
C["hhcode"] = C["hhcode"].astype("str")

In [149]:
# MERGE
years05_06 = pd.merge(B_relevant, C_relevant, on=["hhcode", "idc"], how='inner', indicator=False)

# Evaluate merge 
#years05_06._merge.value_counts() #(ALL GOOD)

In [150]:
# Select people in school enrollment age (I would say 4-10 years old)
years05_06 = years05_06.loc[(years05_06["age"] >= 4)&(years05_06["age"] <= 15)]

In [151]:
years05_06.province.value_counts()
# weird! It doesnt have the provinces with codes 5,6,7

1    13846
2     9127
3     8520
4     6026
Name: province, dtype: int64

In [152]:
# Lets see which hh codes we have:
years05_06["hhcode"] = years05_06["hhcode"].astype("str")
years05_06['subprovince code']= years05_06['hhcode'].apply(lambda x: x[0:4])
years05_06["subprovince code"] = years05_06["subprovince code"].astype("int")
years05_06.loc[(years05_06["region"] == 1), 'region'] = 'urban'
years05_06.loc[(years05_06["region"] == 2), 'region'] = 'rural'

In [153]:
# Now we import the stratum name - code data
strat_name = pd.read_csv(
    "/Users/clarissaache/Documents/IDS 701/stratum0506_v1.csv"
)
strat_name.sample(5)

,province,subprovince code,subprovince,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
89,Balochistan,4162,Gwadar,NaN,NaN,NaN,NaN
62,NWFP,3122,Karak,NaN,NaN,NaN,NaN
44,Sindh,2102,Hyderabad,NaN,NaN,NaN,NaN
138,Balochistan,4061,Nasirabad,NaN,NaN,NaN,NaN
69,NWFP,3192,Batagram,NaN,NaN,NaN,NaN


In [154]:
# MERGE
years05_06_ = pd.merge(years05_06, strat_name, on=["subprovince code"], how='inner', indicator=True)

# Evaluate merge 
years05_06_._merge.value_counts()

both          37519
left_only         0
right_only        0
Name: _merge, dtype: int64

In [155]:
years05_06_.sample(3)

,hhcode,idc,age,sex,marital_status,ever_admitted,max_level_acheived,currently_enrolled,why_not,region,province_x,subprovince code,province_y,subprovince,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,_merge
9837,1212030112.0,4,15,male,unmarried/never married,yes,NaN,yes,NaN,rural,1,1212,Punjab,Okara,NaN,NaN,NaN,NaN,both
19531,2072010202.0,52,6,male,unmarried/never married,yes,NaN,yes,NaN,rural,2,2072,Sindh,Shikarpur,NaN,NaN,NaN,NaN,both
30534,3212040203.0,55,9,female,unmarried/never married,yes,NaN,yes,NaN,rural,3,3212,NWFP,Mardan,NaN,NaN,NaN,NaN,both


In [156]:
# add year
years05_06_['year']=2005
years05_06_['marital_status']=years05_06_['marital_status'].astype('str')

#fix marital status categories
years05_06_.loc[(years05_06_['marital_status'] == "unmarried/never married"), 'marital_status'] = 1
years05_06_.loc[(years05_06_['marital_status'] == "currently married"), 'marital_status'] = 2
years05_06_.loc[(years05_06_['marital_status'] == "nikah solmanised but rukhsati not taken place"), 'marital_status'] = 5
years05_06_.loc[(years05_06_['marital_status'] == "widow/widower"), 'marital_status'] = 3
years05_06_.loc[(years05_06_['marital_status'] == " divorced"), 'marital_status'] = 4
years05_06_['marital_status'].value_counts()
years05_06_['marital_status']=years05_06_['marital_status'].astype('int')

# fix column names
years05_06_['province']=years05_06_['province_y']

In [157]:
#select relevant columns
years05_06_ = years05_06_[
    [
        "hhcode",
        "idc",
        "age",
        "marital_status",
        "sex",
        "ever_admitted",
        "currently_enrolled",
        "region",
        "province",
        "subprovince code",
        "province",
        "year",
    ]
].copy()

In [158]:
years05_06_.to_csv('/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/10_data_cleaning/clean_clarissa/years05_06_.csv', index_label="id")